In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

### Getting the data

In [3]:
path = Path('data')
path.mkdir(parents=True, exist_ok=True)
path

PosixPath('data')

In [4]:
#!wget http://files.fast.ai/part2/lesson14/rossmann.tgz -P {path}

In [5]:
#!tar -xvzf {path}/rossmann.tgz -C {path}
#!rm {path}/rossmann.tgz

In [6]:
#!rm {path}/sample_submission.csv

### Feature engineering

In [7]:
table_paths = sorted(list(path.glob('*.csv')))
table_paths

[PosixPath('data/googletrend.csv'),
 PosixPath('data/state_names.csv'),
 PosixPath('data/store.csv'),
 PosixPath('data/store_states.csv'),
 PosixPath('data/test.csv'),
 PosixPath('data/train.csv'),
 PosixPath('data/weather.csv')]

In [8]:
tables = [pd.read_csv(table_path, low_memory=False) for table_path in table_paths]
googletrend_df, state_names_df, store_df, store_states_df, test_df, train_df, weather_df = tables

#### Train and test data

In [9]:
train_df.shape, test_df.shape

((1017209, 9), (41088, 8))

In [10]:
train_df['Date'] = pd.to_datetime(train_df.Date)

train_df.Date.min(), train_df.Date.max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2015-07-31 00:00:00'))

In [11]:
test_df['Date'] = pd.to_datetime(test_df.Date)

test_df.Date.min(), test_df.Date.max()

(Timestamp('2015-08-01 00:00:00'), Timestamp('2015-09-17 00:00:00'))

In [12]:
train_df.columns.difference(test_df.columns)

Index(['Customers', 'Sales'], dtype='object')

In [13]:
test_df.drop('Id', axis=1, inplace=True)
test_df['Customers'] = np.nan
test_df['Sales'] = np.nan

In [14]:
# Merge train and test to apply preprocessors on both of them at time
df = train_df.append(test_df, sort=False, ignore_index=True, verify_integrity=True)

df.Date.min(), df.Date.max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2015-09-17 00:00:00'))

In [15]:
# 0 and 1 to boolean
df['Open'] = df['Open'] != 0
df['Promo'] = df['Promo'] != 0
df['StateHoliday'] = df['StateHoliday'] != '0' # object
df['SchoolHoliday'] = df['SchoolHoliday'] != 0

In [16]:
import re

def add_datepart(df):
    # Decompose date column into various parts for the purpose of constructing categoricals
    
    date_col = df['Date']
    targ_pre = re.sub('[Dd]ate$', '', 'Date')
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 
            'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    for n in attr: 
        df[targ_pre + n] = getattr(date_col.dt, n.lower())
    df[targ_pre + 'Elapsed'] = date_col.astype(np.int64) // 10 ** 9
    
add_datepart(df)

In [17]:
df.drop('DayOfWeek', axis=1, inplace=True)

In [18]:
df.query('Date==20141231').iloc[0]

Store                                 1
Date                2014-12-31 00:00:00
Sales                              2605
Customers                           327
Open                               True
Promo                             False
StateHoliday                      False
SchoolHoliday                      True
Year                               2014
Month                                12
Week                                  1
Day                                  31
Dayofweek                             2
Dayofyear                           365
Is_month_end                       True
Is_month_start                    False
Is_quarter_end                     True
Is_quarter_start                  False
Is_year_end                        True
Is_year_start                     False
Elapsed                      1419984000
Name: 236380, dtype: object

In [19]:
base_columns = df.columns

#### Store data

In [20]:
def left_outer_join(left, right, left_on, right_on=None, suffixes=('', '_y')):
    # Left outer join which makes unmatched values on the right side zero
    # The suffixes argument describes the naming convention for duplicate fields
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=suffixes)

In [21]:
# Populate stores with state name
store_df = left_outer_join(store_df, store_states_df, "Store")

store_df.State.isnull().sum()

0

In [22]:
store_df['Promo2'] = store_df['Promo2'] != 0

In [23]:
# Merge sales data with store information
df = left_outer_join(df, store_df, "Store")

df.StoreType.isnull().sum()

0

In [24]:
df[df.columns.difference(base_columns)].iloc[0]

Assortment                       a
CompetitionDistance           1270
CompetitionOpenSinceMonth        9
CompetitionOpenSinceYear      2008
Promo2                       False
Promo2SinceWeek                NaN
Promo2SinceYear                NaN
PromoInterval                  NaN
State                           HE
StoreType                        c
Name: 0, dtype: object

In [25]:
base_columns = df.columns

#### Google Trends

In [26]:
store_states_df['State'].unique()

array(['HE', 'TH', 'NW', 'BE', 'SN', 'SH', 'HB,NI', 'BY', 'BW', 'RP',
       'ST', 'HH'], dtype=object)

In [27]:
# Extract date and state for merging purposes
googletrend_df['Date'] = googletrend_df.week.str.split(' - ', expand=True)[0]
googletrend_df['State'] = googletrend_df.file.str.split('_', expand=True)[2]

# Match the rest of the data
googletrend_df.loc[googletrend_df.State=='NI', "State"] = 'HB,NI'

In [28]:
googletrend_df['Date'] = pd.to_datetime(googletrend_df['Date'])

In [29]:
add_datepart(googletrend_df)

In [30]:
# Merge with google trends across states
df = left_outer_join(df, googletrend_df, ["State", "Year", "Week"])

df.trend.isnull().sum()

0

In [31]:
# Merge with google trends across Germany
trend_de_df = googletrend_df[googletrend_df.file == 'Rossmann_DE']
df = left_outer_join(df, trend_de_df, ["Year", "Week"], suffixes=('', '_DE'))

df.trend_DE.isnull().sum()

0

In [32]:
# Remove redundant columns
for c in df.columns:
    if c in ['file', 'week'] or c.endswith('_y') or (c.endswith('_DE') and c != 'trend_DE'):
        if c in df.columns: df.drop(c, inplace=True, axis=1)

In [33]:
df[df.columns.difference(base_columns)].iloc[0]

trend       85
trend_DE    83
Name: 0, dtype: int64

In [34]:
base_columns = df.columns

#### Weather data

In [35]:
weather_df = left_outer_join(weather_df, state_names_df, "file", "StateName")

weather_df.StateName.isnull().sum()

0

In [36]:
weather_df['Date'] = pd.to_datetime(weather_df.Date)

In [37]:
# Merge with weather data
df = left_outer_join(df, weather_df, ["State", "Date"])

df.Mean_TemperatureC.isnull().sum()

0

In [38]:
df.drop(['file', 'StateName'], axis=1, inplace=True)

In [39]:
df[df.columns.difference(base_columns)].iloc[0]

CloudCover                       1
Dew_PointC                       9
Events                         Fog
Max_Gust_SpeedKm_h             NaN
Max_Humidity                    98
Max_Sea_Level_PressurehPa     1021
Max_TemperatureC                23
Max_VisibilityKm                31
Max_Wind_SpeedKm_h              24
MeanDew_PointC                   6
Mean_Humidity                   54
Mean_Sea_Level_PressurehPa    1018
Mean_TemperatureC               16
Mean_VisibilityKm               15
Mean_Wind_SpeedKm_h             11
Min_DewpointC                    3
Min_Humidity                    18
Min_Sea_Level_PressurehPa     1015
Min_TemperatureC                 8
Min_VisibilitykM                10
Precipitationmm                  0
WindDirDegrees                  13
Name: 0, dtype: object

In [40]:
base_columns = df.columns

#### Promo and competition

In [41]:
# Pick an arbitrary signal value that doesn't otherwise appear in the data
df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

In [42]:
from isoweek import Week

df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1))

df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days
df.loc[df.Promo2Days<0, "Promo2Days"] = 0
df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0

df["Promo2Weeks"] = df["Promo2Days"] // 7
df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25 # limit the number of categories

In [43]:
df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)

In [44]:
# Populate some temporal columns related to competition
df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                 month=df.CompetitionOpenSinceMonth, day=15))

df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days
df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0
df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0

df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"] // 30
df.loc[df.CompetitionMonthsOpen<0, "CompetitionMonthsOpen"] = 0
df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24

In [45]:
df[df.columns.difference(base_columns)].iloc[0]

CompetitionDaysOpen                     2510
CompetitionMonthsOpen                     24
CompetitionOpenSince     2008-09-15 00:00:00
Promo2Days                                 0
Promo2Since              1900-01-01 00:00:00
Promo2Weeks                                0
Name: 0, dtype: object

In [46]:
base_columns = df.columns

#### Duration between events

In [47]:
def set_elapsed(df, fld, pre):
    # Given a particular boolean field fld to monitor, 
    # this function will start tracking time since the last occurrence of that field
    
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s, v, d in zip(df.Store.values, df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(abs((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

In [48]:
# Create a temporary dataframe with relevant data
on_columns = ["Date", "Store", "Open", "Promo", "StateHoliday", "SchoolHoliday"]

elapsed_df = df[on_columns].copy()

In [49]:
# Get durations of events
elapsed_df['Closed'] = ~elapsed_df['Open']
event_flds = ['Closed', 'SchoolHoliday', 'StateHoliday', 'Promo']

for fld in event_flds:
    elapsed_df.sort_values(['Store', 'Date'], inplace=True)
    set_elapsed(elapsed_df, fld, 'After')
    elapsed_df.sort_values(['Store', 'Date'], ascending=[True, False], inplace=True)
    set_elapsed(elapsed_df, fld, 'Before')

In [50]:
elapsed_df.set_index("Date", inplace=True)

In [51]:
# Replace NaNs with 0
for suffix in event_flds:
    for prefix in ['Before', 'After']:
        column = prefix + suffix
        elapsed_df[column] = elapsed_df[column].fillna(0).astype(int)

In [52]:
# Calculate the number of events in the previous 7 days
bwd_df = elapsed_df[['Store']+event_flds].sort_index().groupby("Store").rolling(7, min_periods=1).sum()
# _ next 7 days
fwd_df = elapsed_df[['Store']+event_flds].sort_index(ascending=False).groupby("Store").rolling(7, min_periods=1).sum()

In [53]:
# Drop index
bwd_df.drop('Store', axis=1, inplace=True)
bwd_df.reset_index(inplace=True)
fwd_df.drop('Store', axis=1, inplace=True)
fwd_df.reset_index(inplace=True)

In [54]:
bwd_df.columns

Index(['Store', 'Date', 'Closed', 'SchoolHoliday', 'StateHoliday', 'Promo'], dtype='object')

In [55]:
fwd_df.columns

Index(['Store', 'Date', 'Closed', 'SchoolHoliday', 'StateHoliday', 'Promo'], dtype='object')

In [56]:
elapsed_df.columns

Index(['Store', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'Closed',
       'AfterClosed', 'BeforeClosed', 'AfterSchoolHoliday',
       'BeforeSchoolHoliday', 'AfterStateHoliday', 'BeforeStateHoliday',
       'AfterPromo', 'BeforePromo'],
      dtype='object')

In [57]:
elapsed_df = left_outer_join(elapsed_df, bwd_df, ['Date', 'Store'], suffixes=['', '_bw'])
elapsed_df = left_outer_join(elapsed_df, fwd_df, ['Date', 'Store'], suffixes=['', '_fw'])

In [58]:
elapsed_df.sort_values(by=['Store', 'Date']).head(10).T

,989,988,987,986,985,984,983,982,981,980
Date,2013-01-01 00:00:00,2013-01-02 00:00:00,2013-01-03 00:00:00,2013-01-04 00:00:00,2013-01-05 00:00:00,2013-01-06 00:00:00,2013-01-07 00:00:00,2013-01-08 00:00:00,2013-01-09 00:00:00,2013-01-10 00:00:00
Store,1,1,1,1,1,1,1,1,1,1
Open,False,True,True,True,True,False,True,True,True,True
Promo,False,False,False,False,False,False,True,True,True,True
StateHoliday,True,False,False,False,False,False,False,False,False,False
SchoolHoliday,True,True,True,True,True,True,True,True,True,True
Closed,True,False,False,False,False,True,False,False,False,False
AfterClosed,0,1,2,3,4,0,1,2,3,4
BeforeClosed,0,4,3,2,1,0,6,5,4,3
AfterSchoolHoliday,0,0,0,0,0,0,0,0,0,0


In [59]:
# Drop not needed fields before join
elapsed_df.drop(['Open'] + event_flds, axis=1, inplace=True)

In [60]:
df = left_outer_join(df, elapsed_df, ['Store', 'Date'])

In [61]:
df[df.columns.difference(base_columns)].iloc[0]

AfterClosed             5.0
AfterPromo              0.0
AfterSchoolHoliday      0.0
AfterStateHoliday      57.0
BeforeClosed            2.0
BeforePromo             0.0
BeforeSchoolHoliday     0.0
BeforeStateHoliday      0.0
Closed_bw               1.0
Closed_fw               1.0
Promo_bw                5.0
Promo_fw                5.0
SchoolHoliday_bw        5.0
SchoolHoliday_fw        7.0
StateHoliday_bw         0.0
StateHoliday_fw         0.0
Name: 0, dtype: float64

#### Closed stores and zero sales stores

In [62]:
(~df.Open).sum()

178801

In [63]:
(df['Sales'] == 0).sum()

172871

In [64]:
# Closed stores won't be counted into the forecasts
df = df[~((~df.Open) | (df['Sales'] == 0))]

In [65]:
df.drop('Open', axis=1, inplace=True)

#### Save data

In [66]:
df.sort_values(by=['Date', 'Store'], inplace=True)

df.Date.iloc[0], df.Date.iloc[-1]

(Timestamp('2013-01-01 00:00:00'), Timestamp('2015-09-17 00:00:00'))

In [67]:
df.reset_index(inplace=True, drop=True)

df.index

RangeIndex(start=0, stop=879442, step=1)

In [68]:
# Get columns with missing values
df[df.columns[df.isna().any()]].isnull().sum()

Sales                   35104
Customers               35104
CompetitionDistance      2262
PromoInterval          438106
Max_VisibilityKm        22172
Mean_VisibilityKm       22172
Min_VisibilitykM        22172
Max_Gust_SpeedKm_h     677088
CloudCover              73011
Events                 211902
dtype: int64

In [69]:
df[['SchoolHoliday_bw', 'StateHoliday_bw', 'Promo_bw', 'Closed_bw',
    'SchoolHoliday_fw', 'StateHoliday_fw', 'Promo_fw', 'Closed_fw']] = \
df[['SchoolHoliday_bw', 'StateHoliday_bw', 'Promo_bw', 'Closed_bw',
    'SchoolHoliday_fw', 'StateHoliday_fw', 'Promo_fw', 'Closed_fw']].astype('int64')

In [70]:
train_df = df.loc[~df.Sales.isnull()]

train_df.shape

(844338, 76)

In [71]:
train_df[['Sales', 'Customers']] = train_df[['Sales', 'Customers']].astype('int64')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [72]:
train_df.describe(include=[np.number]).loc[['count', 'min', 'max']].T.astype(np.int64)

,count,min,max
Store,844338,1,1115
Sales,844338,46,41551
Customers,844338,8,7388
Year,844338,2013,2015
Month,844338,1,12
Week,844338,1,52
Day,844338,1,31
Dayofweek,844338,0,6
Dayofyear,844338,1,365
Elapsed,844338,1356998400,1438300800


In [73]:
train_df.describe(exclude=[np.number]).T

,count,unique,top,freq,first,last
Date,844338,942,2015-06-09 00:00:00,1115,2013-01-01 00:00:00,2015-07-31 00:00:00
Promo,844338,2,False,467463,NaN,NaN
StateHoliday,844338,2,False,843428,NaN,NaN
SchoolHoliday,844338,2,False,680893,NaN,NaN
Is_month_end,844338,2,False,816299,NaN,NaN
Is_month_start,844338,2,False,824972,NaN,NaN
Is_quarter_end,844338,2,False,835723,NaN,NaN
Is_quarter_start,844338,2,False,836819,NaN,NaN
Is_year_end,844338,2,False,842288,NaN,NaN
Is_year_start,844338,2,False,844285,NaN,NaN


In [74]:
test_df = df.loc[df.Sales.isnull()]
test_df.drop('Sales', axis=1, inplace=True)
test_df.drop('Customers', axis=1, inplace=True)

test_df.shape

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(35104, 74)

In [75]:
# Save final tables
train_df.to_pickle(path/'train_df')
test_df.to_pickle(path/'test_df')